# Context

In [0]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
var defaultVarsBE = Workspace.Query<IfrsVariable>();

# Yield Curve Scenario

In [0]:
var yieldCurve = @"
@@YieldCurve
Currency,Year,Month,Scenario,Values0,Values1,Values2,Values3
EUR,2020,3,,0.008685495,0.008685495,0.018498471,0.019966839
CHF,2020,3,,0.002,0.002,0.002,0.002
CHF,2021,3,,0.1,0.1,0.1,0.1
USD,2021,3,,0.005,0.005,0.005,0.005
USD,2020,12,,0.002,0.002,0.002,0.002
USD,2020,3,,0.002,0.002,0.002,0.002
USD,2020,1,,0.002,0.002,0.002,0.002
EUR,2018,3,MTUP10pct,0.009554045,0.009554045,0.020348318,0.021963523
CHF,2018,3,MTUP10pct,0.0022,0.0022,0.0022,0.0022
CHF,2019,3,MTUP10pct,0.11,0.11,0.11,0.11
USD,2019,3,MTUP10pct,0.0055,0.0055,0.0055,0.0055
USD,2018,12,MTUP10pct,0.0022,0.0022,0.0022,0.0022
USD,2018,3,MTUP10pct,0.0022,0.0022,0.0022,0.0022
USD,2018,1,MTUP10pct,0.0022,0.0022,0.0022,0.0022
EUR,2020,12,MTUP10pct,0.009554045,0.009554045,0.020348318,0.021963523
CHF,2020,12,MTUP10pct,0.0022,0.0022,0.0022,0.0022
CHF,2021,3,MTUP10pct,0.11,0.11,0.11,0.11
USD,2021,3,MTUP10pct,0.0055,0.0055,0.0055,0.0055
USD,2020,12,MTUP10pct,0.0022,0.0022,0.0022,0.0022
USD,2020,3,MTUP10pct,0.0022,0.0022,0.0022,0.0022
USD,2020,1,MTUP10pct,0.0022,0.0022,0.0022,0.0022";

In [0]:
var log = await Import.FromString(yieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log.Warnings.Count().Should().Be(2);

## Test 2020Q4 with Scenario MTUP10pct

In [0]:
var argsBestEstimate = new ImportArgs ("CH",2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var defaultPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate)); 

var scenarioArgs = new ImportArgs ("CH",2020, 12, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(scenarioArgs);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(scenarioArgs);
var diffIfrsVars = Workspace.Query<IfrsVariable>().Except(defaultVarsBE.Where(x => x.Partition == defaultPartition).ToArray(), IfrsVariableComparer.Instance());
diffIfrsVars.ToArray().Length.Should().NotBe(0);

## Test 2021Q1 with Scenario MTUP10pct

In [0]:
var argsBestEstimate = new ImportArgs ("CH",2021, 3, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var defaultPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate)); 

var scenarioArgs = new ImportArgs ("CH",2021, 3, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(scenarioArgs);
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(scenarioArgs);
var diffIfrsVars = Workspace.Query<IfrsVariable>().Except(defaultVarsBE.Where(x => x.Partition == defaultPartition).ToArray(), IfrsVariableComparer.Instance());
diffIfrsVars.ToArray().Length.Should().NotBe(0);

## Robustness: Reimport does not trigger Calculations

In [0]:
var log = await Import.FromString(cashflows_scenarioTest).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
log.Warnings.Count().Should().Be(1);

In [0]:
var extension = System.IO.Path.GetExtension(file);
var stream = await Project.FileStorage.ReadAsync(file);
var mainTab = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet.Tables[Main];",

